## Loading and preprocessing the data

First, we load the raw data using the starter code

In [1]:
def load_data(path, lowercase=True):
    sents = []
    tags = []
    with open(path, 'r', encoding="utf8") as f:
        for line in f.read().splitlines():
            sent = []
            tag = []
            for pair in line.split('####')[1].split(' '):
                tn, tg = pair.rsplit('=', 1)
                if lowercase:
                    sent.append(tn.lower())
                else:
                    sent.append(tn)
                tag.append(tg)
            sents.append(sent)
            tags.append(tag)
    return sents, tags

In [2]:
train_sents, train_tags = load_data("data/twitter1_train.txt")

After loading the raw tweets and tags, we have to clean the data and do a few preprocessing steps before we can start training. PyTorch requires us to label all tokens using positive integers, so in this step we will build a hashmap to help us convert raw sequences of alphanumeric tokens to sequences of integers.

In [3]:
def get_vocab_idx(train):
    tokens = set()
    for sent in train:
        tokens.update(sent)
    tokens = sorted(list(tokens))
    vocab2idx = dict(zip(tokens, range(1, len(tokens)+1)))
    vocab2idx["<PAD>"] = 0
    return vocab2idx

def convert_to_idx(sents, word2idx):
    for sent in sents:
        for i in range(len(sent)):
            sent[i] = word2idx[sent[i]]

In [4]:
vocab2idx = get_vocab_idx(train_sents)
convert_to_idx(train_sents, vocab2idx)

We also need to convert the tags into integers that can then be fed to PyTorch's categorical cross entropy loss function

In [5]:
tag2idx = {"<PAD>": 0, "O": 1, "T-NEG": 2, "T-NEU": 3, "T-POS": 4}
convert_to_idx(train_tags, tag2idx)

## Building the model

In [6]:
import torch
import torch.nn as nn
import torch.nn.functional as F

Any computation model in PyTorch, from a single layer to an entire model, inherits the base class nn.Module. To build our own model, we will need to construct our own class that inherits from nn.Module as well. Other than the constructor function, we will need to fill out the `forward()` function, which takes in the data and returns the output predictions, and will be implicitly called when we feed our data to an instance of our model. 

WARNING: this example below is only to help you understand how to use PyTorch, and is NOT what you will be doing for this homework! It predicts the current label based on only the contextualized representation vector of the current token. In your model, you will be predicting the current label based on the previous label as well as the vector representation of the current token

In [7]:
class EncodeSentence(nn.Module):
    def __init__(self, word_dim, num_words, num_tags):
        super(EncodeSentence, self).__init__()
        self.word_emb = nn.Embedding(num_words, word_dim, padding_idx=0)
        # Output dimension is word_dim//2 because bidirectional doubles the output dimension
        self.contextualizer = nn.LSTM(word_dim, word_dim//2, batch_first=True, bidirectional=True)
        self.output = nn.Linear(word_dim, num_tags)
        
    def forward(self, sent):
        sent_embed = self.word_emb(sent)
        sent_embed, _ = self.contextualizer(sent_embed)
        output_scores = self.output(sent_embed)
        return F.log_softmax(output_scores, dim=1)

PyTorch cannot handle jagged arrays, so if we want to process our input data in batches (with sentences in a batch having different lengths), we have to either pad or trim all sentences to the same size. In this example, I'm padding all sentences in a batch of size 10 so that they all have the same length as the longest sentence.

In [8]:
def pad_sents(sents, pad_idx=0):
    padded_sents = []
    maxlen = max([len(sent) for sent in sents])
    for sent in sents:
        padded_sent = sent.copy()
        padded_sent.extend([pad_idx]*(maxlen-len(sent)))
        padded_sents.append(padded_sent)
    return padded_sents

train_batch = pad_sents(train_sents[:10])
train_label = pad_sents(train_tags[:10])

Now we create an instance of our model, and feed our batch to it to get prediction scores for each label type for each token for all sentences in our batch

In [9]:
model = EncodeSentence(256, len(vocab2idx), len(tag2idx))
scores = model(torch.tensor(train_batch))

## Training the model

To optimize our model parameters, we need to create an instance of any optimizer algorithms provided by PyTorch. Here we're using ADAM. `model.parameters()` will return all the parameters of our model, which will be used by our optimizer instance. If you have more than one model, and you want to optimize all of them end-to-end, then do not forget to feed all of them to the same optimizer!

In [10]:
import torch.optim as optim
optimizer = optim.Adam(list(model.parameters()))

`loss_sum` allows us to accumulate the gradient of all time steps up to `maxlen`, which is the length of the longest sequence in the training batch. This step is necessary because `F.nll_loss()` doesn't allow us to calculate the loss of all time steps simultaneously. If you manage to find a loss function provided by PyTorch that actually calculate categorical softmax loss of the entire sequence in one call, feel free to use it!

In [11]:
maxlen = scores.size(1)
loss_sum = torch.tensor([0.],)
for i in range(maxlen):
    loss_sum += F.nll_loss(scores[:, i, :], torch.tensor(train_label)[:, i])

After we've accumulated the loss at all time steps, backpropagate it through the model to calculate gradient for all parameters, and call `optimizer.step()` to perform a single parameter update througout the model.

In [12]:
loss_sum.backward()
optimizer.step()